In [1]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
gt_df = pd.read_csv('manual_label_result/csvs/labelled_subtitle.csv')
gt_df.head()

,id,subtitle
0,DTfHpK5W4M4-0001929-2.0,但不也是 挺好的嗎
1,QKYCPuoxj6U-0001075-2.0,白虎
2,9cIr5pINQaY-0001757-2.0,NaN
3,aczYWgtYNYw-0002651-2.0,NaN
4,J7Cu0sIACKo-0000566-2.0,NaN


In [3]:
prediction_df = pd.read_csv('manual_label_result/csvs/2018-7-17-10-54.csv')
prediction_df.head()

,id,prediction,confidence
0,7-eVc37Q_w8-0000004-2.0,NaN,-1.000
1,swKJqvSc3ek-0000007-2.0,你找不到人你打電話給我幹什麼,0.990
2,Dj8o7JJSFf0-0000013-2.0,我不是在怪她吧,0.980
3,NCnvTl5jPlQ-0000013-2.0,NaN,-1.000
4,vDpOeD7URIg-0000018-2.0,真希望 得最後決後,0.975


In [4]:
merge_df = gt_df.merge(prediction_df, on='id')
merge_df.head()

,id,subtitle,prediction,confidence
0,DTfHpK5W4M4-0001929-2.0,但不也是 挺好的嗎,但不也是 挺好的嗎,0.975
1,QKYCPuoxj6U-0001075-2.0,白虎,白虎,0.980
2,9cIr5pINQaY-0001757-2.0,NaN,NaN,0.000
3,aczYWgtYNYw-0002651-2.0,NaN,NaN,-1.000
4,J7Cu0sIACKo-0000566-2.0,NaN,NaN,0.000


In [5]:
merge_df['subtitle'] = merge_df.subtitle.str.replace(' ', '')
merge_df['prediction'] = merge_df.prediction.str.replace(' ', '')
merge_df.head()

,id,subtitle,prediction,confidence
0,DTfHpK5W4M4-0001929-2.0,但不也是挺好的嗎,但不也是挺好的嗎,0.975
1,QKYCPuoxj6U-0001075-2.0,白虎,白虎,0.980
2,9cIr5pINQaY-0001757-2.0,NaN,NaN,0.000
3,aczYWgtYNYw-0002651-2.0,NaN,NaN,-1.000
4,J7Cu0sIACKo-0000566-2.0,NaN,NaN,0.000


In [6]:
same_predictoin = merge_df.subtitle==merge_df.prediction
both_nan = merge_df.subtitle.isna() & merge_df.prediction.isna()
merge_df.loc[merge_df.subtitle.isna(),]
print(f"Total images: {merge_df.shape[0]}")
print(f"Total ground truth images containing subtitles: {merge_df.loc[~merge_df.subtitle.isna()].shape[0]}")
print(f"Total predictions matched with groud truth that contain subtitles: {merge_df.loc[same_predictoin].shape[0]}")
print(f"Percentage of matched images containing subtitles {merge_df.loc[same_predictoin].shape[0] / merge_df.loc[~merge_df.subtitle.isna()].shape[0]}")

print(f"Total ground truth images not containing subtitles: {merge_df.loc[merge_df.subtitle.isna()].shape[0]}")
print(f"Total predictions matched with groud truth that doesn't contain subtitle: {merge_df.loc[both_nan].shape[0]}")
print(f"Total correctly predicted images: {merge_df.loc[both_nan | same_predictoin].shape[0]}")

Total images: 996
Total ground truth images containing subtitles: 510
Total predictions matched with groud truth that contain subtitles: 330
Percentage of matched images containing subtitles 0.6470588235294118
Total ground truth images not containing subtitles: 486
Total predictions matched with groud truth that doesn't contain subtitle: 452
Total correctly predicted images: 782


In [23]:
merge_df_filtered = merge_df.loc[~merge_df.prediction.isna()]
merge_df_filtered['label'] = 0
merge_df_filtered.loc[merge_df_filtered.subtitle == merge_df_filtered.prediction, 'label'] = 1
precisions = []
recalls = []
f1s = []
thresholds= []
record_df = pd.DataFrame(columns=['threshold', 'precision', 'recall', 'f1'])
for threshold in [0.5,0.6,0.7,0.8,0.9, 0.95]:
# threshold=0.7
    matched_prediction = merge_df_filtered.subtitle == merge_df_filtered.prediction
    higher_threshold = merge_df_filtered.confidence >= threshold
    merge_df_filtered['predicted_label'] = 0
    merge_df_filtered.loc[higher_threshold, 'predicted_label'] = 1
    recall = recall_score(merge_df_filtered.label, merge_df_filtered.predicted_label)
    precision = precision_score(merge_df_filtered.label.values, merge_df_filtered.predicted_label.values)
    f1 = f1_score(merge_df_filtered.label.values, merge_df_filtered.predicted_label.values)
    recalls.append(recall)
    precisions.append(precision)
    f1s.append(f1)
    thresholds.append(threshold)
record_df['threshold']= thresholds
record_df['precision'] = precisions
record_df['recall'] = recalls
record_df['f1'] = f1s
record_df

/home/A60174/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/A60174/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/A60174/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,threshold,precision,recall,f1
0,0.50,0.674847,1.000000,0.805861
1,0.60,0.691824,1.000000,0.817844
2,0.70,0.709052,0.996970,0.828715
3,0.80,0.727477,0.978788,0.834625
4,0.90,0.786802,0.939394,0.856354
5,0.95,0.827988,0.860606,0.843982


In [27]:
higher_threshold = merge_df_filtered.confidence>0.9
matched_predictions = merge_df_filtered.prediction == merge_df_filtered.subtitle
filtered_samples = merge_df_filtered.loc[higher_threshold].shape[0]
correct_samples = merge_df_filtered.loc[higher_threshold &matched_predictions].shape[0]

print(f"Total filtered sample: {filtered_samples}")
print(f"Total correct sample: {correct_samples}")
print(f"Correct sample rate: {correct_samples/filtered_samples }")

Total filtered sample: 394
Total correct sample: 310
Correct sample rate: 0.7868020304568528
